# Spectral Loss for Fundamental Mode of N3 as outer ring moves closer

Here we compare the spectral loss for the N3 configuration to that of N3 as we decrease the buffer air region of N3. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf
from ngsolve.webgui import Draw
from scipy.optimize import newton

First we load and visualize the spectral loss for N1

# Initialize

In [ ]:
n = 300
wls = np.linspace(1.4e-6, 2e-6, n+1)
outer = 'h2'
nu = 1

N2betas = np.load('data/N2_base.npy')

In [ ]:
N2betas = np.load('data/N2_base.npy')
N3betas = np.load('data/N3base.npy')
N1betas = np.load('../N2_varying_thickness_studies/data/N1_wls.npy')

In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air, lambda x : n_glass, lambda x: n_air]
mats = ['air', 'glass','air', 'glass', 'air']
maxhs = [100 for r in mats]

d = 4.0775e-05
ts = [d , 1e-5, d, 1e-5, 2e-5]
A = BraggExact(ts=ts,ns=ns, mats=mats, maxhs=maxhs)

In [ ]:
# Formula for loss spikes from article

ms = np.arange(11,15,1)

n1 = A.ns[0]  # Inner (core) index
n2 = A.ns[1]  # Cladding index

d = 15*2/3*1e-6

ls = (2 * n1 * d / ms * ((n2/n1)**2 - 1)**.5)  # when n2 depends on ls, need solver
ls

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

ax.plot(wls, -N2betas.imag, linewidth=1.2, label='base_N2')
ax.plot(wls, -N1betas.imag, linewidth=1.2, label='base_N1')
ax.plot(wls, -N3betas.imag, linewidth=1.2, label='base_N3')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')


# Make air region smaller

This is without filtering out the higher mode though, which I need to do, but this gets it start being set up.

In [ ]:
betas1 = np.zeros_like(wls, dtype=complex)

t1 = .75
d = 4.0775e-05
ts = [d , 1e-5, t1 * d, 1e-5, 2e-5]


for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, mats=mats, maxhs=maxhs, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(N3betas[i])
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas1[i] = beta
    

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

ax.plot(wls, -N3betas.imag, linewidth=1.2, label='base_N3')
ax.plot(wls, -betas1.imag, linewidth=1.2, label='t=.75')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')



# More air


In [ ]:
betas2 = np.zeros_like(wls, dtype=complex)

t1 = .25
d = 4.0775e-05
ts = [d , 1e-5, t1 * d, 1e-5, 2e-5]

centers = betas1
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, mats=mats, maxhs=maxhs, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(centers[i])
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.", beta.imag)
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas2[i] = beta
    

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

ax.plot(wls, -N3betas.imag, linewidth=1.2, linestyle='--', label='base_N3')
ax.plot(wls, -betas1.imag, linewidth=1.2, label='t=.75')
ax.plot(wls, -betas2.imag, linewidth=1.2, label='t=.5')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')


# Even more

In [ ]:
betas3 = np.zeros_like(wls, dtype=complex)

t1 = .1
d = 4.0775e-05
ts = [d , 1e-5, t1 * d, 1e-5, 2e-5]

centers = betas2
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, mats=mats, maxhs=maxhs, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(centers[i])
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.", beta.imag)
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas3[i] = beta

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

ax.plot(wls, -N3betas.imag, linewidth=1.2, linestyle='--', label='base_N3')
ax.plot(wls, -betas1.imag, linewidth=1.2, label='t=.75')
ax.plot(wls, -betas2.imag, linewidth=1.2, label='t=.25')
ax.plot(wls, -betas3.imag, linewidth=1.2, label='t=.1')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')



# Closer

In [ ]:
betas4 = np.zeros_like(wls, dtype=complex)

t1 = .01
d = 4.0775e-05
ts = [d , 1e-5, t1 * d, 1e-5, 2e-5]

centers = betas3
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, mats=mats, maxhs=maxhs, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(centers[i])
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.", beta.imag)
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas4[i] = beta

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

ax.plot(wls, -N3betas.imag, linewidth=1.2, linestyle='--', label='base_N3')
ax.plot(wls, -betas1.imag, linewidth=1.2, label='t=.75')
ax.plot(wls, -betas2.imag, linewidth=1.2, label='t=.25')
ax.plot(wls, -betas3.imag, linewidth=1.2, label='t=.1')
ax.plot(wls, -betas4.imag, linewidth=1.2, label='t=.01')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')



# Even closer

In [ ]:
betas5 = np.zeros_like(wls, dtype=complex)

t1 = .001
d = 4.0775e-05
ts = [d , 1e-5, t1 * d, 1e-5, 2e-5]

centers = betas4
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, mats=mats, maxhs=maxhs, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(centers[i].real)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.", beta.imag)
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas5[i] = beta

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

ax.plot(wls, -N3betas.imag, linewidth=1.2, linestyle='--', label='base_N3')
# ax.plot(wls, -betas1.imag, linewidth=1.2, label='t=.75')
# ax.plot(wls, -betas2.imag, linewidth=1.2, label='t=.25')
# ax.plot(wls, -betas3.imag, linewidth=1.2, label='t=.1')
ax.plot(wls, -betas4.imag, linewidth=1.2, label='t=.01')
ax.plot(wls, -betas5.imag, linewidth=1.2, label='t=.001')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')


# Incredibly close

In [ ]:
betas6 = np.zeros_like(wls, dtype=complex)

t1 = .0001
d = 4.0775e-05
ts = [d , 1e-5, t1 * d, 1e-5, 2e-5]

centers = betas5
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, mats=mats, maxhs=maxhs, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(centers[i].real)
    imag = 0
    flag = True
    redcuce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.", beta.imag)
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas6[i] = beta

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

ax.plot(wls, -N3betas.imag, linewidth=1.2, linestyle='--', label='base_N3')
# ax.plot(wls, -betas1.imag, linewidth=1.2, label='t=.75')
# ax.plot(wls, -betas2.imag, linewidth=1.2, label='t=.25')
# ax.plot(wls, -betas3.imag, linewidth=1.2, label='t=.1')
# ax.plot(wls, -betas4.imag, linewidth=1.2, label='t=.01')
ax.plot(wls, -betas5.imag, linewidth=1.2, label='t=.001')
ax.plot(wls, -betas6.imag, linewidth=1.2, label='t=.0001')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')


# Check if this is same ultimate profile as a 2x thick N1

In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air]
mats = ['air', 'glass','air']
maxhs = [100 for r in mats]

d = 4.0775e-05
ts = [d , 2e-5, d]
A = BraggExact(ts=ts,ns=ns, mats=mats, maxhs=maxhs)

In [ ]:
# Formula for loss spikes from article

ms = np.arange(11,15,1)

n1 = A.ns[0]  # Inner (core) index
n2 = A.ns[1]  # Cladding index

d = 15*2/3*1e-6

ls = (2 * n1 * d / ms * ((n2/n1)**2 - 1)**.5)  # when n2 depends on ls, need solver
ls

In [ ]:
betas7 = np.zeros_like(wls, dtype=complex)

d = 4.0775e-05
ts = [d , 2e-5, d]

centers = betas5
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, mats=mats, maxhs=maxhs, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99999*k_low)
    imag = 0
    flag = True
    redcuce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.", beta.imag)
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas7[i] = beta

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

# ax.plot(wls, -N3betas.imag, linewidth=1.2, linestyle='--', label='base_N3')
# ax.plot(wls, -betas1.imag, linewidth=1.2, label='t=.75')
# ax.plot(wls, -betas2.imag, linewidth=1.2, label='t=.25')
# ax.plot(wls, -betas3.imag, linewidth=1.2, label='t=.1')
# ax.plot(wls, -betas4.imag, linewidth=1.2, label='t=.01')
ax.plot(wls, -betas5.imag, linewidth=1.2, label='t=.001')
ax.plot(wls, -betas6.imag, linewidth=1.2, label='t=.0001')
ax.plot(wls, -betas7.imag, linewidth=1.2, label='2x thick N1')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')


# That's pretty convincing

But the distance is still having an effect, the loss is lower, let's get even closer

In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air, lambda x : n_glass, lambda x: n_air]
mats = ['air', 'glass','air', 'glass', 'air']
maxhs = [100 for r in mats]

d = 4.0775e-05
ts = [d , 1e-5, d, 1e-5, 2e-5]
A = BraggExact(ts=ts,ns=ns, mats=mats, maxhs=maxhs)

In [ ]:
betas8 = np.zeros_like(wls, dtype=complex)

t1 = .000001
d = 4.0775e-05
ts = [d , 1e-5,0, 1e-5, 2e-5]

centers = betas6
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, mats=mats, maxhs=maxhs, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(.999999*k_low)
    imag = 0
    flag = True
    redcuce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.", beta.imag)
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas8[i] = beta

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

# ax.plot(wls, -N3betas.imag, linewidth=1.2, linestyle='--', label='base_N3')
# ax.plot(wls, -betas1.imag, linewidth=1.2, label='t=.75')
# ax.plot(wls, -betas2.imag, linewidth=1.2, label='t=.25')
# ax.plot(wls, -betas3.imag, linewidth=1.2, label='t=.1')
# ax.plot(wls, -betas4.imag, linewidth=1.2, label='t=.01')
# ax.plot(wls, -betas5.imag, linewidth=1.2, label='t=.001')
# ax.plot(wls, -betas6.imag, linewidth=1.2, label='t=.0001')
ax.plot(wls, -betas8.imag, linewidth=1.2, label='t=0')
ax.plot(wls, -betas7.imag, linewidth=1.2, label='2x thick N1')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')


# Nope

No matter what it seems, the loss is lower with arbitrarily close layers (that are nonetheless separated).

# In fact, that looked like a bug

I checked it though and it wasn't.  It looks like I'm getting off track at some point with my thickness and catching higher order modes.